In [ ]:
%run crawling.ipynb

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()
from konlpy.tag import Hannanum
hannanum=Hannanum()
from konlpy.tag import Komoran
komoran=Komoran()
from konlpy.tag import Okt
okt=Okt()

In [ ]:
for i in range(len(job_cards)):
    print(job_cards[i]['main_job'])

In [ ]:
len(job_cards)

In [ ]:
job_cards

In [ ]:
from summa import keywords

# 중복을 체크할 set을 만들기
seen = set()

# 각 공고에서 명사 추출 후 중요 단어 계산
important_jobs = []
for job in job_cards:
    important_task = job['main_job']
    
    try:
        # 명사 추출
        nouns = komoran.nouns(important_task)
        
        # 2글자 이상의 명사만 필터링
        filtered_nouns = [noun for noun in nouns if len(noun) > 1]
        
        # 명사 리스트를 문자열로 변환
        text = " ".join(filtered_nouns)
        
        # 중요 단어 추출
        important_keywords = keywords.keywords(text, words=10, split=True)  # 추출 단어 최대 10개 제한
        
        # 중요 단어 처리
        if not important_keywords:  # 중요 단어가 없을 경우 원본 리스트 할당
            job['important_keywords'] = filtered_nouns  # 오류 발생 시 원본 리스트(filtered_nouns) 할당
        else:
            # 중요 단어가 3개 초과일 경우 상위 5개 제한
            important_keywords = [word for phrase in important_keywords for word in phrase.split()]  # 구문을 단어로 분리
            job['important_keywords'] = important_keywords[:10]  # 상위 5개만 사용
    
    except Exception as e:
        # 예외 처리: 오류 발생 시 원본 리스트 할당
        print(f"Company: {job['CO_name']}, Keywords: {filtered_nouns}")
        job['important_keywords'] = filtered_nouns  # 원본 명사 리스트(filtered_nouns) 할당

    # 중복 확인을 위해 (회사명, 키워드) 조합으로 세트에 추가
    keywords_tuple = tuple(job['important_keywords'])
    if (job['CO_name'], keywords_tuple) not in seen:
        important_jobs.append(job)  # 중복되지 않으면 추가
        seen.add((job['CO_name'], keywords_tuple))  # 세트에 추가하여 중복을 방지

# 결과 출력
for job in important_jobs:
    print(f"Company: {job['CO_name']}, Keywords: {job['important_keywords']}")
